In [1]:
import os
from pathlib import Path


while True:
    project_name = input("Enter the Src Folder Name: ")
    if project_name != '':
        break


list_of_files = [
f"src/__init__.py",
f"src/__pycache__",
f"src/{project_name}/__pycache__",
f"src/{project_name}/__init__.py",


f"src/{project_name}/graph/__pycache__",
f"src/{project_name}/graph/__init__.py",
\
f"src/{project_name}/LLMS/__pycache__",
f"src/{project_name}/LLMS/__init__.py",

f"src/{project_name}/nodes/__pycache__",
f"src/{project_name}/nodes/__init__.py",

f"src/{project_name}/state/__pycache__",
f"src/{project_name}/state/__init__.py",

f"src/{project_name}/tools/__init__.py",

f"src/{project_name}/user_interface/__pycache__",
f"src/{project_name}/user_interface/__init__.py",

f"src/{project_name}/user_interface/streamlitui/__pycache__",

f"src/{project_name}/vector_store/__init__.py",
]

for filepath in list_of_files:
    filepath=Path(filepath)
    filedir, filename=os.path.split(filepath)
    
    if filedir != "":
        os.makedirs(filedir,exist_ok=True)
    if not os.path.exists(filepath):
        with open(filepath,"w") as f:
            pass

In [2]:
list_of_files

['src/__init__.py',
 'src/__pycache__',
 'src/Langgraph_Agentic_AI/__pycache__',
 'src/Langgraph_Agentic_AI/__init__.py',
 'src/Langgraph_Agentic_AI/graph/__pycache__',
 'src/Langgraph_Agentic_AI/graph/__init__.py',
 'src/Langgraph_Agentic_AI/LLMS/__pycache__',
 'src/Langgraph_Agentic_AI/LLMS/__init__.py',
 'src/Langgraph_Agentic_AI/nodes/__pycache__',
 'src/Langgraph_Agentic_AI/nodes/__init__.py',
 'src/Langgraph_Agentic_AI/state/__pycache__',
 'src/Langgraph_Agentic_AI/state/__init__.py',
 'src/Langgraph_Agentic_AI/tools/__init__.py',
 'src/Langgraph_Agentic_AI/user_interface/__pycache__',
 'src/Langgraph_Agentic_AI/user_interface/__init__.py',
 'src/Langgraph_Agentic_AI/user_interface/streamlitui/__pycache__',
 'src/Langgraph_Agentic_AI/vector_store/__init__.py']

In [3]:
%%writefile  src/Langgraph_Agentic_AI/user_interface/uiconfigfile.ini
[DEFAULT]
PAGE_TITLE = LangGraph: Build Stateful Agentic AI graph
LLM_OPTIONS = Groq
USECASE_OPTIONS = Basic Chatbot
GROQ_MODEL_OPTIONS = qwen-2.5-32b, mixtral-8x7b-32768, llama3-8b-8192, llama3-70b-8192, gemma-7b-i

Writing src/Langgraph_Agentic_AI/user_interface/uiconfigfile.ini


In [4]:
%%writefile  src/Langgraph_Agentic_AI/user_interface/uiconfigfile.py
from configparser import ConfigParser

class Config:
    def __init__(self,config_file="./src/Langgraph_Agentic_AI/user_interface/uiconfigfile.ini"):
        self.config=ConfigParser()
        self.config.read(config_file)

    def get_llm_options(self):
        return self.config["DEFAULT"].get("LLM_OPTIONS").split(", ")
    
    def get_usecase_options(self):
        return self.config["DEFAULT"].get("USECASE_OPTIONS").split(", ")

    def get_groq_model_options(self):
        return self.config["DEFAULT"].get("GROQ_MODEL_OPTIONS").split(", ")

    def get_page_title(self):
        return self.config["DEFAULT"].get("PAGE_TITLE")

  

Writing src/Langgraph_Agentic_AI/user_interface/uiconfigfile.py


In [5]:
%%writefile  src/Langgraph_Agentic_AI/state/state.py
from typing import Annotated, Literal, Optional
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, AIMessage

class State(TypedDict):
    """
    Represents the structure of the state used in the graph.
    """
    messages: Annotated[list, add_messages]

Writing src/Langgraph_Agentic_AI/state/state.py


In [6]:
%%writefile  src/Langgraph_Agentic_AI/LLMS/llm.py
import os
import streamlit as st
from langchain_groq import ChatGroq

class GroqLLM:
    def __init__(self,user_controls_input):
        self.user_controls_input=user_controls_input

    def get_llm_model(self):
        try:
            groq_api_key=self.user_controls_input['GROQ_API_KEY']
            selected_groq_model=self.user_controls_input['selected_groq_model']
            if groq_api_key=='' and os.environ["GROQ_API_KEY"] =='':
                st.error("Please Enter the Groq API KEY")

            llm = ChatGroq(api_key =groq_api_key, model=selected_groq_model)

        except Exception as e:
            raise ValueError(f"Error Occurred with Exception : {e}")
        return llm

Writing src/Langgraph_Agentic_AI/LLMS/llm.py


In [7]:
%%writefile  src/Langgraph_Agentic_AI/nodes/basic_chatbot_node.py
from src.Langgraph_Agentic_AI.state.state import State

class BasicChatbotNode:
    """
    Basic chatbot logic implementation.
    """
    def __init__(self,model):
        self.llm = model

    def process(self, state: State) -> dict:
        """
        Processes the input state and generates a chatbot response.
        """
        return {"messages":self.llm.invoke(state['messages'])}

Writing src/Langgraph_Agentic_AI/nodes/basic_chatbot_node.py


In [8]:
%%writefile src/Langgraph_Agentic_AI/graph/graph_builder.py
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate

from src.Langgraph_Agentic_AI.state.state import State
from src.Langgraph_Agentic_AI.nodes.basic_chatbot_node import BasicChatbotNode

class GraphBuilder:

    def __init__(self,model):
        self.llm=model
        self.graph_builder=StateGraph(State)

    def basic_chatbot_build_graph(self):
        """
        Builds a basic chatbot graph using LangGraph.
        This method initializes a chatbot node using the `BasicChatbotNode` class 
        and integrates it into the graph. The chatbot node is set as both the 
        entry and exit point of the graph.
        """
        self.basic_chatbot_node=BasicChatbotNode(self.llm)
        self.graph_builder.add_node("chatbot",self.basic_chatbot_node.process)
        self.graph_builder.add_edge(START,"chatbot")
        self.graph_builder.add_edge("chatbot",END)

    def setup_graph(self, usecase: str):
        """
        Sets up the graph for the selected use case.
        """
        if usecase == "Basic Chatbot":
            self.basic_chatbot_build_graph()
        return self.graph_builder.compile()

Writing src/Langgraph_Agentic_AI/graph/graph_builder.py


In [9]:
%%writefile src/Langgraph_Agentic_AI/user_interface/streamlitui/loadui.py
import streamlit as st
import os
from datetime import date

from langchain_core.messages import AIMessage,HumanMessage
from src.Langgraph_Agentic_AI.user_interface.uiconfigfile import Config


class LoadStreamlitUI:
    def __init__(self):
        self.config =  Config() # config
        self.user_controls = {}

    def initialize_session(self):
        return {
        "current_step": "requirements",
        "requirements": "",
        "user_stories": "",
        "po_feedback": "",
        "generated_code": "",
        "review_feedback": "",
        "decision": None
    }
  


    def load_streamlit_ui(self):
        st.set_page_config(page_title= self.config.get_page_title(), layout="wide")
        st.header(self.config.get_page_title())
        st.session_state.timeframe = ''
        st.session_state.IsFetchButtonClicked = False
        st.session_state.IsSDLC = False
        
        

        with st.sidebar:
            # Get options from config
            llm_options = self.config.get_llm_options()
            usecase_options = self.config.get_usecase_options()

            # LLM selection
            self.user_controls["selected_llm"] = st.selectbox("Select LLM", llm_options)

            if self.user_controls["selected_llm"] == 'Groq':
                # Model selection
                model_options = self.config.get_groq_model_options()
                self.user_controls["selected_groq_model"] = st.selectbox("Select Model", model_options)
                # API key input
                self.user_controls["GROQ_API_KEY"] = st.session_state["GROQ_API_KEY"] = st.text_input("API Key",
                                                                                                      type="password")
                # Validate API key
                if not self.user_controls["GROQ_API_KEY"]:
                    st.warning("⚠️ Please enter your GROQ API key to proceed. Don't have? refer : https://console.groq.com/keys ")
                   
            
            # Use case selection
            self.user_controls["selected_usecase"] = st.selectbox("Select Usecases", usecase_options)
            
            if "state" not in st.session_state:
                st.session_state.state = self.initialize_session()  
        
        return self.user_controls

Writing src/Langgraph_Agentic_AI/user_interface/streamlitui/loadui.py


In [10]:
%%writefile src/Langgraph_Agentic_AI/user_interface/streamlitui/display_result.py
import streamlit as st
from langchain_core.messages import HumanMessage,AIMessage
import json

class DisplayResultStreamlit:
    def __init__(self,usecase,graph,user_message):
        self.usecase= usecase
        self.graph = graph
        self.user_message = user_message

    def display_result_on_ui(self):
        usecase= self.usecase
        graph = self.graph
        user_message = self.user_message
        if usecase =="Basic Chatbot":
                for event in graph.stream({'messages':("user",user_message)}):
                    print(event.values())
                    for value in event.values():
                        print(value['messages'])
                        with st.chat_message("user"):
                            st.write(user_message)
                        with st.chat_message("assistant"):
                            st.write(value["messages"].content)

Writing src/Langgraph_Agentic_AI/user_interface/streamlitui/display_result.py


In [11]:
%%writefile src/Langgraph_Agentic_AI/main.py
import streamlit as st
import json
from src.Langgraph_Agentic_AI.user_interface.streamlitui.loadui import LoadStreamlitUI
from src.Langgraph_Agentic_AI.user_interface.streamlitui.display_result import DisplayResultStreamlit

from src.Langgraph_Agentic_AI.LLMS.llm import GroqLLM
from src.Langgraph_Agentic_AI.graph.graph_builder import GraphBuilder

# MAIN Function START
def load_langgraph_agenticai_app():
    """
    Loads and runs the LangGraph AgenticAI application with Streamlit UI.
    This function initializes the UI, handles user input, configures the LLM model,
    sets up the graph based on the selected use case, and displays the output while 
    implementing exception handling for robustness.
    """
   
    # Load UI
    ui = LoadStreamlitUI()
    user_input = ui.load_streamlit_ui()

    if not user_input:
        st.error("Error: Failed to load user input from the UI.")
        return

    # Text input for user message
    if st.session_state.IsFetchButtonClicked:
        user_message = st.session_state.timeframe 
    else :
        user_message = st.chat_input("Enter your message:")

    if user_message:
            try:
                # Configure LLM
                obj_llm_config = GroqLLM(user_controls_input=user_input)
                model = obj_llm_config.get_llm_model()
                
                if not model:
                    st.error("Error: LLM model could not be initialized.")
                    return

                # Initialize and set up the graph based on use case
                usecase = user_input.get('selected_usecase')
                if not usecase:
                    st.error("Error: No use case selected.")
                    return
                

                ### Graph Builder
                graph_builder=GraphBuilder(model)
                try:
                    graph = graph_builder.setup_graph(usecase)
                    DisplayResultStreamlit(usecase,graph,user_message).display_result_on_ui()
                except Exception as e:
                    st.error(f"Error: Graph setup failed - {e}")
                    return
                

            except Exception as e:
                 raise ValueError(f"Error Occurred with Exception : {e}")

Writing src/Langgraph_Agentic_AI/main.py


In [12]:
%%writefile app.py
from src.Langgraph_Agentic_AI.main import load_langgraph_agenticai_app


if __name__=="__main__":
    load_langgraph_agenticai_app()

Overwriting app.py


In [1]:
!streamlit run app.py

^C
